In [ ]:
import pandas as pd

#### Загрузить данные в pandas по url=https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv

In [ ]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
df = pd.read_csv(url, sep = "\t")

#### 1. Вывести: кол-во наблюдений в датасете

In [ ]:
print(len(df))

In [ ]:
print(df)

df_modified - содержит все элементы без and

In [ ]:
df_and_only = df[df["item_name"].str.find("and") != -1]
df_modified = df.drop(df_and_only.index).reset_index()
df_modified = df_modified.drop('index', axis=1)
df_chips = df[df["item_name"] == "Chips"].drop_duplicates().head(1)
chips_price = df_chips.item_price.values[0]
df_chips = df_and_only.copy()
df_chips["item_name"] = "Chips"
df_chips["item_price"] = chips_price
df_sause = df_and_only.copy()
df_sause["item_name"] = df_sause.apply(lambda x: x["item_name"][10:], axis=1)
df_sause["item_price"] = df_sause["item_price"] - chips_price
df_and_only = pd.concat([df_chips, df_sause], ignore_index=True)
df_modified = pd.concat([df_modified, df_and_only], ignore_index=True)

#### 2. Вывести названия столбцов

In [ ]:
print(df.columns.values) # df.columns.tolist()

#### 5. Измените тип переменной item_price c с помощью лямбды функции

 Item_sum = item_price * quantity

In [ ]:
df["item_sum"] = df.apply(lambda x: float(x["item_price"][1:-1]), axis=1) 
df["item_price"] = df.apply(lambda x: float(x["item_sum"])/x["quantity"], axis=1) 

#### 3. Определить самую частую позицию (item) в заказе 

In [ ]:
# unique_df = df.groupby(["order_id", "item_name"]).count().reset_index()
# unique_df = unique_df[unique_df["quantity"] >= 2][["order_id", "item_name","quantity"]]
# correct_df = df.loc[~df["order_id"].isin(unique_df["order_id"])]
# incorrect_df = df.loc[df["order_id"].isin(unique_df["order_id"]), ['item_name', 'choice_description', 'item_price']].drop_duplicates()

In [ ]:
print(df_modified.groupby("item_name").agg({"quantity":"sum"}).sort_values("quantity", ascending=False)) #.head(1).index.values) # либо tolist()

#### 4. Построить гистрограмму частоты заказов по позициям (item )

Ограничимся 10 наиболее часто заказываемыми позициями

In [ ]:
import matplotlib.pyplot as plt

df_hist = df_modified.groupby("item_name", as_index = False).agg({"quantity":"sum"}).sort_values("quantity", ascending=False).head(10)
plt.barh(df_hist.item_name.values, df_hist.quantity.values)
plt.xlabel("Number of orders")

#### 6. Построить гистограмму кол-во денег заработанных по каждой позиции (item)

In [ ]:
df_hist_sum = df_modified.groupby("item_name", as_index = False).agg({"item_sum":"sum"}).sort_values("item_sum", ascending=False).head(10)
plt.barh(df_hist_sum.item_name.values, df_hist_sum.item_sum.values)
plt.xlabel("Sum in dollars")

#### 7. Средняя сумма заказа? (минимум 2 способа)

In [ ]:
print(df.groupby("order_id").agg({"item_sum":"sum"}).mean())
sum = df["item_sum"].sum()
num = len(df["order_id"].unique())
print(sum/num)

#### 8. Выведите среднее, минимальное и максимальное, медианное значения позиций в заказе

In [ ]:
print(df.groupby("order_id").agg({"item_price":["min", "max", "mean", "median"]}))

#### 9. Определить статистику заказов стейков, а также статистику заказов прожарки.

- df_steak - содержит все блюда, содержашие в своем имени слово "Steak". 
- В полях "mild", "medium", "hot" указывается сколько раз упоминалось это слово в "choice description". 
- В поле "list_fire" содержится СПИСОК всех упоминаний "mild", "medium", "hot"

In [ ]:
import re
import numpy as np
dishes_lst = df["item_name"].unique().tolist()
steak_dishes_lst = list(filter(lambda x: x.find("Steak") != -1, dishes_lst))
df_steak = df[df["item_name"].isin(steak_dishes_lst)].reset_index().drop('index', axis=1)
print("Колличество заказов стейков ", df_steak["item_name"].count())
# print(df.loc[df["item_name"].isin(steak_lst)].agg({"order_id":"count", "quantity":"sum"}))
# res = df_steak.loc[df_steak["item_name"].isin(steak_dishes_lst), ["choice_description"]]["choice_description"].unique().tolist()
# for i in res:
#     print(re.findall(r'\((\w+)\)', i))
# (Medium), (Mild), (Hot)


def find_type(x, type):
    res = re.findall(r'\({}\)'.format(type), x["choice_description"])
    return len(res) if res else np.nan

def find_all_types(x):
    res = re.findall(r'\((\w+)\)', x["choice_description"])
    return res if res else np.nan


df_steak['mild'] = df_steak.apply(lambda x: find_type(x, "Mild"), axis=1)
df_steak['medium'] = df_steak.apply(lambda x: find_type(x, "Medium"), axis=1)
df_steak['hot'] = df_steak.apply(lambda x: find_type(x, "Hot"), axis=1)
df_steak = df_steak.astype({'mild': "Int64", 'medium': "Int64", 'hot': "Int64"})
df_steak['list_fire'] = df_steak.apply(lambda x: find_all_types(x), axis=1)
print("Колличество заказов стейков Mild ", df_steak["mild"].count())
print("Колличество заказов стейков Medium ", df_steak["medium"].count())
print("Колличество заказов стейков Hot ", df_steak["hot"].count())

#### 10. Добавить новый столбец цен на каждую позицию в заказе в рублях.

In [ ]:
df["item_price_rub"]= df.apply(lambda x: x["item_price"]*71, axis=1)
print(df)

#### 11. Сгруппировать заказы по входящим позициям в него. Отдельно сгруппировать по стейкам во всех видах прожарках.

In [ ]:
print(df_modified.groupby("item_name").agg({"order_id":"count"}))
print(df_steak.astype('str').groupby(['list_fire']).agg({"order_id":"count"})) #[df_steak.list_fire.notna()]

#### 12. Определить цену по каждой позиции в отдельности. 

In [ ]:
df_unique_item = df_modified.groupby("item_name", as_index=False).agg({"item_price":lambda x: list(set(x.values.tolist()))})
print(df_unique_item)

### Алгоритмы уменьшения размерности 

#### повторить эксперименты в главе 5.1 Qalitative Comparison of Multiple Algorithms статьи статьи UMAP: Uniform Manifold
#### Approximation and Projection for Dimension Reduction (https://arxiv.org/pdf/1802.03426.pdf)

#### необходимо воспроизвести все графики данной главы, соотнести выводы из статьи с вашими результатами.